## Part 1. Coordinates files avilable in server

In [1]:
'''
# to get all the files aviliable in  server
get 2 lists of file names from the CNT server

cd /mnt/leif/littlab/data/Human_Data/CNT_iEEG_BIDS
'ls' > table2_rid_files_avilable_in_server.csv

cd /mnt/leif/littlab/data/Human_Data/recon/BIDS_penn/
'ls' > table3_redcap_rid_to_hupid.csv

# upload table2_rid_files_avilable_in_server.csv and table3_redcap_rid_to_hupid.csv to colab
'''

"\n# to get all the files aviliable in  server\nget 2 lists of file names from the CNT server\n\ncd /mnt/leif/littlab/data/Human_Data/CNT_iEEG_BIDS\n'ls' > table2_rid_files_avilable_in_server.csv\n\ncd /mnt/leif/littlab/data/Human_Data/recon/BIDS_penn/\n'ls' > table3_redcap_rid_to_hupid.csv\n\n# upload table2_rid_files_avilable_in_server.csv and table3_redcap_rid_to_hupid.csv to colab\n"

## Part 2. Format RedCAP HUPID to RID file

In [2]:
# import csv
import pandas as pd
table0= pd.read_excel('table0_master_pt_list.xlsx')  # ieeg.org
table1 = pd.read_csv('table1_redcap_rid_to_hupid.csv')  # obtained from REDCap that contain rid hupid conversion
table2 = pd.read_csv('table2_rid_files_avilable_in_server.csv')  # obtained from CNT server
table3 = pd.read_csv('table3_rid_files_avilable_in_server.csv')  # obtained from CNT server; another directory possbily containing the patient rid coordinate file

# add a header to table2 and table3
table2.columns = ['rid_filename']
table3.columns = ['rid_filename']

In [3]:
## format table1

# change the "Record ID" column in table1 to string instead of int
table1['Record ID'] = table1['Record ID'].astype(str)

# convert HUP Number to HUPID with prefix 'HUP'; convert Record ID to table1
# add 0 or 00 in front of the table1 so there are four digits
table1['HUPID'] = 'HUP' + table1['HUP Number'].astype(str)
for i in range(len(table1)):
    if len(table1['Record ID'][i]) == 2:
      #table1['Record ID'][i] = '00' + table1['Record ID'][i]
      table1.loc[i, 'Record ID'] = '00' + table1['Record ID'][i]
    if len(table1['Record ID'][i]) == 3:
      #table1['Record ID'][i] = '0' + table1['Record ID'][i]
      table1.loc[i, 'Record ID'] = '0' + table1['Record ID'][i]
    else:
      #table1['Record ID'][i] = table1['Record ID'][i]
      table1.loc[i, 'Record ID'] = table1['Record ID'][i]

# rename Record ID column to table1; drop the HUP Number column
table1 = table1.rename(columns={'Record ID': 'RID'})
table1 = table1.drop(columns=['HUP Number'])

# add in rid_filename columns
table1['rid_filename'] = 'sub-RID' + table1['RID']

# check
table1.head()

,RID,HUPID,rid_filename
0,0623,HUP035,sub-RID0623
1,0624,HUP036,sub-RID0624
2,0625,HUP037,sub-RID0625
3,0626,HUP038,sub-RID0626
4,0627,HUP039,sub-RID0627


In [4]:
# merge rid with hupid table on the HUPID column
table01 = pd.merge(table0, table1, on='HUPID')
table01 = table01[['HUPID','RID','rid_filename']]
table01.head()
len_table01 = len(table01)
print('There are', len_table01, 'number of RID ID that can be corresponded to HUP ID.')

There are 58 number of RID ID that can be corresponded to HUP ID.


## Part 3. Merge the two lists of avilable coordinates files with HUPID-RID conversion table

In [5]:
# merge table01 with table2 on the rid_filename column
table012 = pd.merge(table01, table2, on='rid_filename')
len_table012 = len(table012)

# merge table01 with table3 on the rid_filename column
table013 = pd.merge(table01, table3, on='rid_filename')
len_table013 = len(table013)

print('Directory 1 contains', len_table012, 'files')
print('Directory 2 contains', len_table013, 'files')

Directory 1 contains 14 files
Directory 2 contains 42 files


In [6]:
# add rid_dir column to table012
table012['rid_dir'] = '/mnt/leif/littlab/data/Human_Data/CNT_iEEG_BIDS/' + table012['rid_filename'] + '/derivatives/ieeg_recon/module3'

# add rid_dir column to table013
table013['rid_dir'] = '/mnt/leif/littlab/data/Human_Data/recon/BIDS_penn/' + table013['rid_filename'] + '/derivatives/ieeg_recon/module3'

# eliminate HUPID in table012 if it also appear in table013
# table012 = table012[~table012['HUPID'].isin(table013['HUPID'])]

# concatenate table012 and table013 vertically.
table0123 = pd.concat([table012, table013], axis=0).reset_index(drop=True)

# check
table0123.head()
print('There are', len(table0123), 'number of coordinate files avilable in the two directories in server.')

There are 56 number of coordinate files avilable in the two directories in server.


In [7]:
# save csv
csv_filename = '/content/table4_coord_files_dir.csv'
table0123.to_csv(csv_filename, index=False)

## Part 4. Given the directories to the coordinate files, extract them to local directory

In [8]:
# to move all the coord files avilable on remote server to my data directory for later matlab processing

'''
#!/bin/bash

# Define the local directory (the directory where the script is run)
LOCAL_DIR=$(pwd)

# Read the CSV file, skip the header line
tail -n +2 table4_coord_files_dir.csv | while IFS=, read -r HUPID RID rid_filename rid_dir
do
  # Remove any surrounding quotes and whitespace
  rid_dir=$(echo "$rid_dir" | tr -d '"' | xargs)
  HUPID=$(echo "$HUPID" | tr -d '"' | xargs)

  # Check if the directory exists
  if [ -d "$rid_dir" ]; then
    # Change to the directory
    cd "$rid_dir" || continue

    # Find files containing 'atlas-atropos' and '.csv'
    for file in *atlas-atropos*.csv; do
      # Check if the file exists
      if [ -f "$file" ]; then
        # Copy and rename the file to the local directory
        cp "$file" "$LOCAL_DIR/${HUPID}.csv"

        # Process the copied file in the local directory
        cd "$LOCAL_DIR" || exit

        # Remove quotes and whitespace, keep only name, x, y, z columns
        awk -F',' '
        BEGIN {OFS=","}
        NR==1 {
            # Find indices of desired columns
            for (i=1; i<=NF; i++) {
                gsub(/"/, "", $i)
                if ($i == "name") n=i
                else if ($i == "x") x=i
                else if ($i == "y") y=i
                else if ($i == "z") z=i
            }
            print "name","x","y","z"
        }
        NR>1 {
            # Remove single quotes, double quotes, and whitespace from desired fields
            gsub(/["[:space:]]/, "", $n)
            gsub(/["[:space:]]/, "", $x)
            gsub(/["[:space:]]/, "", $y)
            gsub(/["[:space:]]/, "", $z)
            print $n, $x, $y, $z
        }
        ' "${HUPID}.csv" > "${HUPID}_processed.csv"

        # Replace the original file with the processed file
        mv "${HUPID}_processed.csv" "${HUPID}.csv"

        # Return to the original directory
        cd "$rid_dir" || exit

      fi
    done

    # Return to the local directory
    cd "$LOCAL_DIR" || exit
  else
    echo "Directory $rid_dir does not exist."
  fi
done
'''

'\n#!/bin/bash\n\n# Define the local directory (the directory where the script is run)\nLOCAL_DIR=$(pwd)\n\n# Read the CSV file, skip the header line\ntail -n +2 table4_coord_files_dir.csv | while IFS=, read -r HUPID RID rid_filename rid_dir\ndo\n  # Remove any surrounding quotes and whitespace\n  rid_dir=$(echo "$rid_dir" | tr -d \'"\' | xargs)\n  HUPID=$(echo "$HUPID" | tr -d \'"\' | xargs)\n\n  # Check if the directory exists\n  if [ -d "$rid_dir" ]; then\n    # Change to the directory\n    cd "$rid_dir" || continue\n\n    # Find files containing \'atlas-atropos\' and \'.csv\'\n    for file in *atlas-atropos*.csv; do\n      # Check if the file exists\n      if [ -f "$file" ]; then\n        # Copy and rename the file to the local directory\n        cp "$file" "$LOCAL_DIR/${HUPID}.csv"\n\n        # Process the copied file in the local directory\n        cd "$LOCAL_DIR" || exit\n\n        # Remove quotes and whitespace, keep only name, x, y, z columns\n        awk -F\',\' \'\n        B

## Part 5. Report missing csv

In [9]:
# compare the list of coordinates files that we actually extracted from the server with table4, which is the coordinates files we would expected to be extracted from the server
# report the files that are missing
'''
cd /users/skylerz
'ls' > /users/skylerz/table5_output.csv
'''

"\ncd /users/skylerz\n'ls' > /users/skylerz/table5_output.csv\n"

In [28]:
table4 = pd.read_csv('table4_coord_files_dir.csv')
table5 = pd.read_csv('table5_output.csv')

table5.columns = ['HUPID']
table5['HUPID'] = table5['HUPID'].str.replace('.csv', '') # remove the .csv suffix in table5 HUPID column

# get a list of HUPID that existed in table4 but not in table5
table45_missing = table4[~table4['HUPID'].isin(table5['HUPID'])].reset_index(drop=True)
table45_missing

,HUPID,RID,rid_filename,rid_dir
0,HUP211,650,sub-RID0650,/mnt/leif/littlab/data/Human_Data/CNT_iEEG_BID...
1,HUP219,695,sub-RID0695,/mnt/leif/littlab/data/Human_Data/CNT_iEEG_BID...
2,HUP211,650,sub-RID0650,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...
3,HUP219,695,sub-RID0695,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...
4,HUP222,782,sub-RID0782,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...
5,HUP237,885,sub-RID0885,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...
6,HUP251,923,sub-RID0923,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...
7,HUP284,1086,sub-RID1086,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...
8,HUP286,1098,sub-RID1098,/mnt/leif/littlab/data/Human_Data/recon/BIDS_p...


In [35]:
csv_filename = '/content/table6_missing_coord_files.csv'
table45_missing.to_csv(csv_filename, index=False)